<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="1000" alt="cognitiveclass.ai logo"  />
</center>

# Assignment: Exploratory Data Analysis with SQL

Estimated time needed: 60 minutes

# Introduction

Using this R notebook you will perform exploratory data analysis tasks using SQL queries with the RODBC R package. You will not be graded here, but you will be asked to present some of these results in your final assignment, where you will be graded on the accuracy of your results as well as the content of your SQL queries.


### Establish your Db2 connection

Load the 'RODBC' library, and use the 'odbcConnect( )' function as you did in the previous lab to establish the connection to your Db2 assets.\
Provided you successfully loaded your data into the tables in that lab, you are now ready to start running SQL queries using the RODBC library as you did in Course 3.


In [1]:
# provide your solution here
library(RODBC)
# Establish database connection
dsn_driver <- "{IBM DB2 ODBC Driver}"
dsn_database <- "bludb"            # e.g. "bludb"
dsn_hostname <- "ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud" # e.g "54a2f15b-5c0f-46df-8954-.databases.appdomain.cloud"
dsn_port <- "31321"   # e.g. "32733" 
dsn_protocol <- "TCPIP"            # i.e. "TCPIP"
dsn_uid <- "mnx90803"        # e.g. "zjh17769"
dsn_pwd <- "4oVsC7tOEMNqHnRV"      # e.g. "zcwd4+8gbq9bm5k4"  
dsn_security <- "ssl"

conn_path <- paste("DRIVER=",dsn_driver,
              ";DATABASE=",dsn_database,
              ";HOSTNAME=",dsn_hostname,
              ";PORT=",dsn_port,
              ";PROTOCOL=",dsn_protocol,
              ";UID=",dsn_uid,
              ";PWD=",dsn_pwd,
              ";SECURITY=",dsn_security,        
                sep="")
conn <- odbcDriverConnect(conn_path)

# Dump connection info
##############################################################
sql.info <- sqlTypeInfo(conn)
conn.info <- odbcGetInfo(conn)
conn.info["DBMS_Name"]
conn.info["DBMS_Ver"]
conn.info["Driver_ODBC_Ver"]

DBMS_Name 
"DB2/LINUXX8664"

DBMS_Ver 
"11.05.0600"

Driver_ODBC_Ver 
        "03.51"

## Task 1 - Record Count

#### Determine how many records are in the seoul_bike_sharing dataset.


### Solution 1


In [2]:
# provide your solution here
query = "SELECT COUNT(*) AS NUM_OF_RECORDS FROM SEOUL_BIKE_SHARING"
sqlQuery(conn, query)

,NUM_OF_RECORDS
,<int>
1,8465


## Task 2 - Operational Hours

#### Determine how many hours had non-zero rented bike count.


### Solution 2


In [3]:
# provide your solution here
query = "SELECT COUNT(*) AS NUM_OF_HOURS FROM SEOUL_BIKE_SHARING WHERE RENTED_BIKE_COUNT<>0"
sqlQuery(conn, query)

,NUM_OF_HOURS
,<int>
1,8465


## Task 3 - Weather Outlook

#### Query the the weather forecast for Seoul over the next 3 hours.

Recall that the records in the CITIES_WEATHER_FORECAST dataset are 3 hours apart, so we just need the first record from the query.


### Solution 3


In [4]:
# provide your solution here
query = "SELECT * FROM CITIES_WEATHER_FORECAST 
WHERE CITY='Seoul' AND FORECAST_DATETIME=(SELECT MAX(FORECAST_DATETIME) FROM CITIES_WEATHER_FORECAST)"
sqlQuery(conn, query)

,CITY,WEATHER,VISIBILITY,TEMP,TEMP_MIN,TEMP_MAX,PRESSURE,HUMIDITY,WIND_SPEED,WIND_DEG,SEASON,FORECAST_DATETIME
,<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<fct>,<dttm>
1,Seoul,Clouds,10000,25.1,25.1,25.1,1020,17,1.54,262,Spring,2021-04-21 09:00:00


## Task 4 - Seasons

#### Find which seasons are included in the seoul bike sharing dataset.


### Solution 4


In [5]:
# provide your solution here
query = "SELECT DISTINCT SEASONS FROM SEOUL_BIKE_SHARING"
sqlQuery(conn, query)

,SEASONS
,<fct>
1,Autumn
2,Spring
3,Summer
4,Winter


## Task 5 - Date Range

#### Find the first and last dates in the Seoul Bike Sharing dataset.


### Solution 5


In [6]:
# provide your solution here
query = "SELECT MIN(DATE) AS FIRST_DATE, MAX(DATE) AS LAST_DATE FROM SEOUL_BIKE_SHARING"
sqlQuery(conn, query)

,FIRST_DATE,LAST_DATE
,<fct>,<fct>
1,2017-01-12,2018-12-11


## Task 6 - Subquery - 'all-time high'

#### determine which date and hour had the most bike rentals.


### Solution 6


In [7]:
# provide your solution here
query = "SELECT DATE, HOUR, RENTED_BIKE_COUNT FROM SEOUL_BIKE_SHARING 
WHERE RENTED_BIKE_COUNT=(SELECT MAX(RENTED_BIKE_COUNT) FROM SEOUL_BIKE_SHARING)"
sqlQuery(conn, query)

,DATE,HOUR,RENTED_BIKE_COUNT
,<fct>,<int>,<int>
1,2018-06-19,18,3556


## Task 7 - Hourly popularity and temperature by season

#### Determine the average hourly temperature and the average number of bike rentals per hour over each season. List the top ten results by average bike count.


### Solution 7


In [8]:
# provide your solution here
query = "SELECT SEASONS, AVG(RENTED_BIKE_COUNT) AS AVG_RENTALS, AVG(TEMPERATURE) AS AVG_TEMP 
FROM SEOUL_BIKE_SHARING GROUP BY SEASONS"
sqlQuery(conn, query)

,SEASONS,AVG_RENTALS,AVG_TEMP
,<fct>,<int>,<dbl>
1,Autumn,924,13.821167
2,Spring,746,13.021389
3,Summer,1034,26.587274
4,Winter,225,-2.540463


## Task 8 - Rental Seasonality

#### Find the average hourly bike count during each season.

Also include the minimum, maximum, and standard deviation of the hourly bike count for each season.


### Solution 8


In [9]:
# provide your solution here
query = "SELECT SEASONS, AVG(RENTED_BIKE_COUNT) AS AVG_COUNT, MIN(RENTED_BIKE_COUNT) AS MIN_COUNT, 
MAX(RENTED_BIKE_COUNT) AS MAX_COUNT, STDDEV(RENTED_BIKE_COUNT) AS STDDEV_COUNT 
FROM SEOUL_BIKE_SHARING GROUP BY SEASONS"
sqlQuery(conn, query)

,SEASONS,AVG_COUNT,MIN_COUNT,MAX_COUNT,STDDEV_COUNT
,<fct>,<int>,<int>,<int>,<dbl>
1,Autumn,924,2,3298,617.3885
2,Spring,746,2,3251,618.5247
3,Summer,1034,9,3556,690.0884
4,Winter,225,3,937,150.3374


Let's explore a bit and see what might be the most significant contributing factors in terms of the provided data.


## Task 9 - Weather Seasonality

#### Consider the weather over each season. On average, what were the TEMPERATURE, HUMIDITY, WIND_SPEED, VISIBILITY, DEW_POINT_TEMPERATURE, SOLAR_RADIATION, RAINFALL, and SNOWFALL per season?

Include the average bike count as well , and rank the results by average bike count so you can see if it is correlated with the weather at all.


### Solution 9


In [10]:
# provide your solution here
query = "SELECT SEASONS, AVG(TEMPERATURE) AS AVG_TEMP, AVG(HUMIDITY) AS AVG_HUMID, AVG(WIND_SPEED) AS AVG_WIND, 
AVG(VISIBILITY) AS AVG_VIS, AVG(DEW_POINT_TEMPERATURE) AS AVG_DEW_POINT, AVG(SOLAR_RADIATION) AS AVG_RADIATION,
AVG(RAINFALL) AS AVG_RAIN, AVG(SNOWFALL) AS AVG_SNOW, AVG(RENTED_BIKE_COUNT) AS AVG_COUNT
FROM SEOUL_BIKE_SHARING GROUP BY SEASONS"
sqlQuery(conn, query)

,SEASONS,AVG_TEMP,AVG_HUMID,AVG_WIND,AVG_VIS,AVG_DEW_POINT,AVG_RADIATION,AVG_RAIN,AVG_SNOW,AVG_COUNT
,<fct>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,Autumn,13.821167,59,1.492101,1558,5.150594,0.5227827,0.11765617,0.06350026,924
2,Spring,13.021389,58,1.857778,1240,4.091389,0.6803009,0.18694444,0.00000000,746
3,Summer,26.587274,64,1.609420,1501,18.750136,0.7612545,0.25348732,0.00000000,1034
4,Winter,-2.540463,49,1.922685,1445,-12.416667,0.2981806,0.03282407,0.24750000,225


## Task 10 - Total Bike Count and City Info for Seoul

#### Use an implicit join across the WORLD_CITIES and the BIKE_SHARING_SYSTEMS tables to determine the total number of bikes avaialble in Seoul, plus the following city information about Seoul: CITY, COUNTRY, LAT, LON, POPULATION, in a single view.

Notice that in this case, the CITY column will work for the WORLD_CITIES table, but in general you would have to use the CITY_ASCII column.


### Solution 10


In [11]:
# provide your solution here
query = "SELECT W.CITY, W.COUNTRY, W.LAT, W.LNG, W.POPULATION, B.BICYCLES FROM WORLD_CITIES W, BIKE_SHARING_SYSTEMS B
WHERE W.CITY='Seoul' AND W.CITY=B.CITY"
sqlQuery(conn, query)

,CITY,COUNTRY,LAT,LNG,POPULATION,BICYCLES
,<fct>,<fct>,<dbl>,<dbl>,<int>,<int>
1,Seoul,"Korea, South",37.58,127,21794000,20000


## Task 11 - Find all city names and coordinates with comparable bike scale to Seoul's bike sharing system

#### Find all cities with total bike counts between 15000 and 20000. Return the city and country names, plus the coordinates (LAT, LNG), population, and number of bicycles for each city.

Later we will ask you to visualize these similar cities on leaflet, with some weather data.


### Solution 11


In [12]:
# provide your solution here
query = "SELECT W.CITY, W.COUNTRY, W.LAT, W.LNG, W.POPULATION, B.BICYCLES FROM WORLD_CITIES W, BIKE_SHARING_SYSTEMS B
WHERE B.BICYCLES>=15000 AND B.BICYCLES<='20000' AND W.CITY=B.CITY"
sqlQuery(conn, query)

,CITY,COUNTRY,LAT,LNG,POPULATION,BICYCLES
,<fct>,<fct>,<dbl>,<dbl>,<int>,<int>
1,Shanghai,China,31.16,121.46,22120000,19165
2,Seoul,"Korea, South",37.58,127.00,21794000,20000
3,Beijing,China,39.90,116.39,19433000,16000
4,Weifang,China,36.71,119.10,9373000,20000
5,Ningbo,China,29.87,121.54,7639000,15000
6,Zhuzhou,China,27.84,113.14,3855609,20000


In [13]:
close(conn)

## Author(s)

<h4> Jeff Grossman </h4>

## Contributor(s)

<h4> Yan Luo, Rav Ahuja </h4>

## Change log

| Date       | Version | Changed by    | Change Description                  |
| ---------- | ------- | ------------- | ----------------------------------- |
| 2021-05-04 | 0.2     | Jeff Grossman | Finalize                            |
| 2021-04-22 | 0.2     | Jeff Grossman | Update as per peer review           |
| 2021-04-10 | 0.2     | Jeff Grossman | Add data sources and implicit joins |
| 2021-04-05 | 0.1     | Jeff Grossman | Start content creation              |

## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
